In [15]:
from gbif_utils import *
from geo_utils import *
import pandas as pd
import requests
import json
import plotly.graph_objects as go
import geopy.distance
import plotly.express as px
import numpy as np
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
import datetime


In [16]:
GRID_RES = 16
BASE_TAXON_ID = 212
RADIUS=20
center = (47.53660, 7.61344)
now = datetime.datetime.now().strftime("%Y-%m-%d")
DATE_RANGE = ("2022",now)
HARD_POINT_LIMIT = 100000
points = get_min_max_coordinates(center, RADIUS)
lats, lons = get_lat_lon_cells(points,GRID_RES)

In [17]:
center_marker = go.Scattermapbox(
    lon=[center[1]],
    lat=[center[0]],
    text=["center"],
    mode="markers",
    marker={"size": 20, "color": "red", "opacity": 1},
)


def get_marker(point):
    return go.Scattermapbox(
        lon=[point[1]],
        lat=[point[0]],
        mode="markers",
        marker={"size": 10, "opacity": 0.8},
    )


def get_rectangle(points):
    lat_max = max([p[0] for p in points])
    lat_min = min([p[0] for p in points])
    lon_max = max([p[1] for p in points])
    lon_min = min([p[1] for p in points])
    return go.Scattermapbox(
        lon=[lon_min, lon_max, lon_max, lon_min, lon_min],
        lat=[lat_max, lat_max, lat_min, lat_min, lat_max],
        mode="lines",
        marker={"color": "#69DB7C"},
    )


def get_gridlines(lats, lons):
    lat_max = max(lats)
    lat_min = min(lats)
    lon_max = max(lons)
    lon_min = min(lons)
    lines_lat_v = []
    lines_lon_v = []
    lines_lat_h = []
    lines_lon_h = []
    for lo in lons:
        lines_lat_v.append(lat_min)
        lines_lat_v.append(lat_max)
        lines_lat_v.append(None)
        lines_lon_v.append(lo)
        lines_lon_v.append(lo)
        lines_lon_v.append(None)
    for lo in lats:
        lines_lon_h.append(lon_min)
        lines_lon_h.append(lon_max)
        lines_lon_h.append(None)
        lines_lat_h.append(lo)
        lines_lat_h.append(lo)
        lines_lat_h.append(None)
    return go.Scattermapbox(
        lon=lines_lon_v + lines_lon_h,
        lat=lines_lat_v + lines_lat_h,
        mode="lines",
        marker={"color": "deeppink"},
    )


swisstopo_layer = {
    "below": "traces",
    "sourcetype": "raster",
    "sourceattribution": '<a href="http://osm.org/copyright">SwissTopo</a>',
    "source": [
        "https://wmts10.geo.admin.ch/1.0.0/ch.swisstopo.swissimage/default/current/3857/{z}/{x}/{y}.jpeg"
    ],
}
fig = go.Figure(data=[center_marker, get_rectangle(points)])

fig.add_trace(get_gridlines(lats, lons))
fig.update_layout(
    margin=dict(l=0, r=0, t=0, b=0),
    width=600,
    height=600,
    mapbox={
        "style": "white-bg",
        "zoom": 9.5,
        "center": {
            "lat": 47.53660,
            "lon": 7.61344,
        },
        "layers": [
            swisstopo_layer,
        ],
    },
    showlegend=False,
)


In [18]:

def plot_cluster_counts(lats, lons, cpc):
    ct = []
    x = []
    y = []
    for i in range(1, len(lats)):
        for j in range(1, len(lons)):
            ct.append(cpc[i][j])
            x.append(lons[j - 1] + (lons[j] - lons[j - 1]) / 2)
            y.append(lats[i - 1] + (lats[i] - lats[i - 1]) / 2)

    return go.Figure(
        go.Scatter(
            x=x,
            y=y,
            #marker_color=ct,
            marker_color="teal",
            text=ct,
            marker_size=[max(s / (max(ct) / 50), 2) for s in ct],
            mode="markers",
        )
    ).update_layout(
        height=600, width=600, margin=dict(l=0, t=0, b=0, r=0), template="plotly_white"
    )

def get_cpc(lats,lons):
    cpc = [[0 for i in range(len(lons))] for i in range(len(lats))]
    def get_counts(i, j):
        nr = get_number_of_occurencies(
            BASE_TAXON_ID,
            decimal_latitude=(lats[i - 1], lats[i]),
            decimal_longitude=(lons[j - 1], lons[j]),
            date_range=DATE_RANGE,
        )
        return i, j, nr

    with tqdm(total=(len(lats)-1)*(len(lons)-1)) as pbar:
        with ThreadPoolExecutor(max_workers=None) as ex:

            futures = []
            for i in range(1, len(lats)):
                for j in range(1, len(lons)):
                    futures.append(
                        ex.submit(
                            get_counts, i, j
                        )
                    )
            for future in as_completed(futures):
                i, j, ct = future.result()
                cpc[i][j] = ct
                if ct >= HARD_POINT_LIMIT:
                    print("Too many Points!")
                pbar.update(1)

    if np.max(cpc) >= HARD_POINT_LIMIT:
        print("TOO MANY POINTS IN CELL!")
    print("Total observations",np.sum(cpc))
    return cpc

def get_unique_species_keys(lats,lons,cpc):
    usk = []
    for i in range(1, len(lats)):
        print(i,":",lats[i-1], " - ", lats[i])
        with tqdm(total=len(cpc[i])) as pbar:
            with ThreadPoolExecutor(max_workers=None) as ex:

                futures = []
                for j in range(1, len(lons)):
                    if cpc[i][j] > 0:
                        futures.append(
                            ex.submit(
                                get_species_keys_from_occurences,
                                BASE_TAXON_ID,
                                decimal_latitude=(lats[i - 1], lats[i]),
                                decimal_longitude=(lons[j - 1], lons[j]),
                                date_range=DATE_RANGE,
                                unique=True,
                            )
                        )
                for future in as_completed(futures):
                    usk += future.result()
                    usk = list(set(usk))
                    pbar.update(1)
    return usk

def get_species_infos_from_usk(usk):
    species_all_infos = []
    with tqdm(total=len(usk)) as pbar:
        with ThreadPoolExecutor(max_workers=None) as ex:

            futures = []
            for i in range(len(usk)):
                futures.append(
                    ex.submit(
                        get_species_info,
                        usk[i]
                    )
                )
            for future in as_completed(futures):
                res = future.result()
                if res is not None:
                    species_all_infos.append(res)
                pbar.update(1)
    return species_all_infos

In [19]:
cpc = get_cpc(lats,lons)
print(np.sum(cpc), np.max(cpc))
plot_cluster_counts(lats,lons,cpc)

100%|██████████| 256/256 [00:12<00:00, 20.83it/s]

Total observations 49323
49323 10555


In [20]:
grid_coordinates = get_grid_coordinates(lats, lons)
occ = []
with tqdm(total=np.sum(cpc)) as pbar:
    with ThreadPoolExecutor(max_workers=None) as executor:
        thread_results = [
            executor.submit(
                get_occurences,
                BASE_TAXON_ID,
                date_range=DATE_RANGE,
                decimal_latitude=gc.get("lat"),
                decimal_longitude=gc.get("lon"),
            )
            for gc in grid_coordinates
        ]
        for future in as_completed(thread_results):
            try:
                res = future.result()
                if isinstance(res, list):

                    pbar.update(len(res))
                    occ += res
            except:
                print("exc.")
occ = update_dataset_names(occ)
occ = [trim_strings(o) for o in occ]
len(occ)

 67%|██████▋   | 33004/49323 [00:18<00:11, 1446.14it/s]

invalid date 2022-04-31T00:00:00


100%|█████████▉| 49322/49323 [00:24<00:00, 2035.51it/s]


49322

1. Create a copy of `credentials_example.py` :
    ```sh
    cp credentials_example.py credentials.py
    ```
2. Edit the file and insert the credentials


In [21]:
from credentials import host, port, user, password, database
import psycopg2
from psycopg2.extras import execute_values, execute_batch

In [22]:
def postgresql_connect():
    try:
        conn = psycopg2.connect(
            host=host,
            port=port,
            user=user,
            password=password,
            database=database,
        )
        return conn
    except Exception as e:
        print(e)

conn = postgresql_connect()

def get_postgres_version():
    cursor = conn.cursor()
    cursor.execute("SELECT version()")
    version = cursor.fetchone()
    cursor.close()
    return version[0]

def execute_query(query, args=None, conn=conn, df=True):
    try:
        cur = conn.cursor()
        if args:
            cur.execute(query, args)
        else:
            cur.execute(query)

        res = cur.fetchall()
        if df:
            cols = []
            for elt in cur.description:
                cols.append(elt[0])
            df = pd.DataFrame(data=res, columns=cols)
            return df
        else:
            return res
    except Exception as e:
        return e

def execute_query_commit(query, args, conn=conn):
    cur = conn.cursor()
    if args:
        cur.execute(query, args)
    conn.commit()
    conn.reset()

get_postgres_version()

'PostgreSQL 14.7 (Debian 14.7-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit'

In [23]:
count_before = execute_query("select count(*) from gbif").values[0][0]
print(count_before, "values in db")
conn.reset()

418061 values in db


In [24]:
query = """
    INSERT INTO public.gbif (
        "key", eventDate, decimalLongitude, decimalLatitude,
        taxonKey, kingdomKey, phylumKey, classKey, orderKey, familyKey, genusKey, speciesKey,
        "references", gbifReference,
        datasetKey, datasetName, datasetReference, license,
        basisOfRecord,
        mediaType, media
    ) VALUES (
        %(key)s, %(eventDate)s, %(decimalLongitude)s, %(decimalLatitude)s,
        %(taxonKey)s, %(kingdomKey)s, %(phylumKey)s, %(classKey)s, %(orderKey)s,
        %(familyKey)s, %(genusKey)s, %(speciesKey)s, %(references)s, %(gbifReference)s,
        %(datasetKey)s, %(datasetName)s, %(datasetReference)s, %(license)s,
        %(basisOfRecord)s, %(mediaType)s, %(media)s
    ) ON CONFLICT DO NOTHING
    """
cur = conn.cursor()

execute_batch(cur, query, occ, page_size=1000
              )
conn.commit()

In [25]:
count_after = execute_query("select count(*) from gbif").values[0][0]
print(count_after, "values in db")
print(count_after-count_before, "new datapoints")

463844 values in db
45783 new datapoints
